### Version compatibility check

This notebook compares the xsnow package installed in your environment with the documentation version it was written for. The helper below calls `scripts/check_docs_version.py` so you can confirm that the package and docs align before continuing.


In [ ]:
from __future__ import annotations

import subprocess
import sys
from pathlib import Path
import warnings


def _find_script() -> Path | None:
    current = Path.cwd().resolve()
    for candidate in [current, *current.parents]:
        script = candidate / "scripts" / "check_docs_version.py"
        if script.exists():
            return script
    return None


def get_docs_version() -> tuple[str | None, str | None]:
    script_path = _find_script()
    if script_path is None:
        return None, "scripts/check_docs_version.py was not found"
    try:
        completed = subprocess.run(
            [sys.executable, str(script_path)],
            check=True,
            capture_output=True,
            text=True,
        )
    except subprocess.CalledProcessError as exc:
        output = (exc.stdout or "") + (exc.stderr or "")
        return None, output.strip() or str(exc)
    return completed.stdout.strip() or None, None


docs_version, docs_error = get_docs_version()

try:
    import xsnow
    package_version = xsnow.__version__
except Exception as exc:  # pylint: disable=broad-except
    xsnow = None  # type: ignore[assignment]
    package_version = None
    package_error = str(exc)
else:
    package_error = None

print(f"xsnow package version: {package_version if package_version else 'not installed'}")
if package_error and not package_version:
    print(f"Import error: {package_error}")

if docs_version:
    print(f"xsnow docs version: {docs_version}")
else:
    message = "xsnow docs version: unavailable"
    if docs_error:
        message += f" ({docs_error})"
    print(message)

if docs_version and package_version and docs_version != package_version:
    warnings.warn(
        "xsnow package version differs from the documentation version. "
        "Consider aligning them before executing the notebook.",
        stacklevel=2,
    )

# 06: Extending xsnow

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Austfi/xsnowForPatrol/blob/main/notebooks/06_extending_xsnow.ipynb)

This notebook explores xsnow's extension system and shows you how to create custom analysis functions.

## What You'll Learn

- Understanding xsnow's architecture
- The extension system and how it works
- Creating custom computed variables
- Registering new methods
- Contributing to xsnow
- Best practices for extensions


## Installation (For Colab Users)

Set `INSTALL_XSNOW = True` in the next cell if you need to install xsnow. When enabled you can pick `INSTALL_METHOD = "pip"` to install published packages or `INSTALL_METHOD = "dev"` to work from a local clone. The cell also installs the supporting scientific Python stack used throughout the course.


In [ ]:
import subprocess
import sys
from pathlib import Path

INSTALL_XSNOW = False  # Set to True to install or update xsnow in this environment.
INSTALL_METHOD = "pip"  # Choose "pip" for a package install, or "dev" for a developer clone.
DEV_REPO_URL = "https://gitlab.com/avacollabra/postprocessing/xsnow.git"
DEV_CLONE_DIR = Path.home() / "xsnow-dev"


def _run(cmd: list[str]) -> None:
    print(f"$ {' '.join(cmd)}")
    subprocess.check_call(cmd)


try:
    import xsnow
    print(f"xsnow {xsnow.__version__} is already available.")
except Exception as exc:  # pylint: disable=broad-except
    xsnow = None  # type: ignore[assignment]
    print(f"xsnow is not currently available: {exc}")
    if not INSTALL_XSNOW:
        print("Set INSTALL_XSNOW = True and re-run this cell to install xsnow (pip or dev clone).")
    else:
        try:
            if INSTALL_METHOD == "pip":
                _run([sys.executable, "-m", "pip", "install", "--quiet", "numpy", "pandas", "xarray", "matplotlib", "seaborn", "dask", "netcdf4"])
                _run([sys.executable, "-m", "pip", "install", "--quiet", "git+https://gitlab.com/avacollabra/postprocessing/xsnow"])
            elif INSTALL_METHOD == "dev":
                if not DEV_CLONE_DIR.exists():
                    _run(["git", "clone", DEV_REPO_URL, str(DEV_CLONE_DIR)])
                _run([sys.executable, "-m", "pip", "install", "--quiet", "-e", str(DEV_CLONE_DIR)])
            else:
                raise ValueError(f"Unsupported INSTALL_METHOD: {INSTALL_METHOD}")
        except subprocess.CalledProcessError as install_error:
            raise RuntimeError("xsnow installation command failed") from install_error
        import xsnow  # noqa: F401  # pylint: disable=import-outside-toplevel
        print(f"xsnow {xsnow.__version__} installed successfully.")
else:
    INSTALL_XSNOW = INSTALL_XSNOW  # no-op so variable is defined for later cells

In [ ]:
import xsnow
import xarray as xr
import numpy as np



## Part 1: Understanding xsnow's Architecture

xsnow uses **composition** rather than inheritance:
- `xsnowDataset` contains an `xarray.Dataset` (accessible via `.data`)
- Methods are forwarded to the underlying xarray object
- Extensions can add new methods via a plugin system

### Key Design Principles

1. **Composition over inheritance**: More flexible and maintainable
2. **Method forwarding**: Unknown methods are passed to xarray
3. **Extension system**: New functionality added via decorators/plugins
4. **Automatic wrapping**: Operations return new `xsnowDataset` objects


In [ ]:
# Explore xsnowDataset structure
import xsnow

# Check if we can inspect the class

# Check for extension registration mechanism
if hasattr(xsnow, 'register_xsnowDataset_method'):
else:


## Part 2: Creating Custom Computed Variables

The simplest extension is adding a new computed variable. You can do this without the extension system:


In [ ]:
# Load sample data
import xsnow

try:
    ds = xsnow.single_profile()
    print("✅ Sample data loaded")
except Exception as e:
    print(f"❌ Error: {e}")
    print("Install: pip install git+https://gitlab.com/avacollabra/postprocessing/xsnow")
    ds = None


## Part 3: Creating Extension Methods

To create a method that can be called on `xsnowDataset` (like `ds.my_custom_method()`), you typically use xsnow's extension system. The exact mechanism depends on xsnow's implementation.

### Example Extension Function

Here's a template for creating an extension:


In [ ]:
# Example: Custom method to compute 24-hour new snow
def compute_hn24(ds):
    """
    Compute 24-hour new snow (HN24) from snow height changes.
    
    This is a simplified example. Real implementation would handle
    edge cases and different time frequencies.
    """
    if 'HS' not in ds.data_vars:
        raise ValueError("HS (snow height) required for HN24 calculation")
    
    # Compute 24-hour difference
    # This assumes hourly data - adjust for your time frequency
    hs_diff = ds['HS'].diff(dim='time', n=24)  # 24 time steps difference
    
    # Only positive changes (snow accumulation)
    hn24 = hs_diff.where(hs_diff > 0, 0)
    
    # Add to dataset
    result = ds.assign(HN24=hn24)
    
    return result

Extension function template:

def compute_hn24(ds):
    \"\"\"Compute 24-hour new snow.\"\"\"
    # Your calculation here
    new_var = ds['HS'].diff(dim='time', n=24)
    return ds.assign(HN24=new_var)

# To register (if extension system available):
# @xsnow.register_xsnowDataset_method
# def compute_hn24(ds):
#     ...
""")


### Using the Extension

Once registered (or as a standalone function):


In [ ]:
# Example usage (if registered as method):
# ds = ds.compute_hn24()

# Or as standalone function:
# ds = compute_hn24(ds)

Using extensions:

# If registered as method:
ds = ds.compute_hn24()

# Or as function:
ds = compute_hn24(ds)

# Or add directly:
ds = ds.assign(HN24=ds['HS'].diff(dim='time', n=24))
""")


## Part 4: Extension Registration

The exact registration mechanism depends on xsnow's implementation. Check xsnow documentation or source code for the current method.

### Common Patterns

1. **Decorator pattern**: `@register_xsnowDataset_method`
2. **Plugin system**: Import extension modules
3. **Monkey patching**: Add methods directly (not recommended)

### Best Practices

- **Document your extensions**: Clear docstrings
- **Handle edge cases**: Check for required variables, handle NaNs
- **Preserve metadata**: Maintain units and attributes
- **Return xsnowDataset**: Ensure compatibility
- **Test thoroughly**: Verify with different datasets


In [ ]:
# Example: Well-documented extension function
def compute_custom_metric(ds, threshold=100.0):
    """
    Compute a custom metric based on density threshold.
    
    Parameters
    ----------
    ds : xsnowDataset
        Input dataset
    threshold : float, optional
        Density threshold in kg/m³ (default: 100.0)
    
    Returns
    -------
    xsnowDataset
        Dataset with new 'custom_metric' variable
    
    Notes
    -----
    This is a simplified example. Real metrics would use
    validated formulas from snow science literature.
    """
    if 'density' not in ds.data_vars:
        raise ValueError("density variable required")
    
    # Compute metric (example: count of layers above threshold)
    above_threshold = (ds['density'] > threshold).sum(dim='layer')
    
    # Add with metadata
    result = ds.assign(
        custom_metric=above_threshold,
        custom_metric_attrs={
            'units': 'count',
            'long_name': f'Layers with density > {threshold} kg/m³',
            'threshold': threshold
        }
    )
    
    return result



## Part 5: Contributing to xsnow

If you create useful extensions, consider contributing them to xsnow:

### Contribution Steps

1. **Check xsnow repository**: Find the GitHub/GitLab repository
2. **Review contribution guidelines**: Read CONTRIBUTING.md
3. **Create extension module**: Follow xsnow's structure
4. **Write tests**: Ensure your extension works correctly
5. **Submit pull request**: Share your work with the community

### What Makes a Good Extension

- **Useful**: Solves a real problem
- **Well-tested**: Works with various datasets
- **Documented**: Clear usage examples
- **Compatible**: Works with xsnow's architecture
- **General**: Applicable to multiple use cases


In [ ]:
To contribute to xsnow:

1. Visit xsnow repository (check xsnow.avacollabra.org for link)
2. Fork the repository
3. Create your extension in appropriate module
4. Follow xsnow's coding style and conventions
5. Add tests
6. Update documentation
7. Submit pull request

Benefits of contributing:
- Help the community
- Get feedback on your code
- Learn from other contributors
- Make xsnow more powerful
""")


## Part 6: Advanced Extension Example

Here's a more complex example that adds a new dimension (like hazard chart extensions do):


In [ ]:
# Example: Extension that adds a new dimension
# This is more advanced and shows the pattern used by hazard charts

def compute_multi_variant_analysis(ds, variants=['mean', 'max', 'min']):
    """
    Compute analysis with multiple variants (adds new dimension).
    
    This pattern is used by hazard chart extensions.
    """
    results = []
    
    for variant in variants:
        if variant == 'mean':
            result = ds['density'].mean(dim='layer')
        elif variant == 'max':
            result = ds['density'].max(dim='layer')
        elif variant == 'min':
            result = ds['density'].min(dim='layer')
        else:
            continue
        
        results.append(result)
    
    # Stack into new dimension
    # This is simplified - real implementation would be more complex
    stacked = xr.concat(results, dim='variant')
    stacked = stacked.assign_coords(variant=variants)
    
    # Add to dataset (this is a simplified approach)
    # Real extensions might create a new view or sub-dataset
    
    
    return stacked



## Summary

✅ **What we learned:**

1. **xsnow architecture**: Composition-based design with method forwarding
2. **Custom variables**: Use `.assign()` to add computed variables
3. **Extension functions**: Create reusable analysis functions
4. **Registration**: Use xsnow's extension system (check docs for exact method)
5. **Best practices**: Documentation, validation, metadata preservation
6. **Contributing**: How to share extensions with the community

## Key Concepts

- **Extension system**: Allows adding methods to xsnowDataset
- **Computed variables**: Add new data variables with `.assign()`
- **Method forwarding**: xsnow forwards unknown methods to xarray
- **Plugin architecture**: Extensions can be loaded dynamically

## Next Steps

- **Create your own extension**: Start with a simple computed variable
- **Check xsnow source**: Learn from existing extensions
- **Contribute**: Share useful extensions with the community
- **Document**: Write clear examples for others

## Resources

- xsnow documentation: https://xsnow.avacollabra.org
- xsnow repository: Check documentation for link
- xarray documentation: https://docs.xarray.dev (for underlying functionality)

## Exercises

1. Create a function to compute a custom stability metric
2. Add a new variable to a dataset using `.assign()`
3. Write a well-documented extension function
4. Check xsnow source code to see how built-in extensions work
5. Plan an extension you'd like to contribute
